In [1]:
!nvidia-smi

Sun May 12 04:30:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


### Fine-tuning으로 고객사별 맞춤 Chatbot 학습 (2) - 학습하고 대화하기

##### 1. Dataset 준비

In [3]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

model_id = "beomi/polyglot-ko-12.8b-safetensors"
tokenizer = AutoTokenizer.from_pretrained(model_id)
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

##### 2. 모델 로드

In [5]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

model-00002-of-00028.safetensors:   0%|          | 0.00/843M [00:00<?, ?B/s]

model-00003-of-00028.safetensors:   0%|          | 0.00/843M [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

In [6]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


##### 3. 모델 학습

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.511000
20,2.205100
30,2.277300
40,2.166500
50,2.180900


TrainOutput(global_step=50, training_loss=2.268168067932129, metrics={'train_runtime': 236.5606, 'train_samples_per_second': 0.423, 'train_steps_per_second': 0.211, 'total_flos': 2101890763407360.0, 'train_loss': 2.268168067932129, 'epoch': 0.004726791453961051})

##### 4. 학습한 모델 사용해보기

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1156: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1542: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34,    6,    6,    6,
            6,    6,    6,    6,    6,    6,    6,    6]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('아침에 먹으면 좋은 과일은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:543: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1542: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 아침에 먹으면 좋은 과일은?

### 답변: 우리는 흔히 아침은 거르고, 저녁에는 조금 많이 먹는 경우가 있어서 속이 불편하게 되거나 속이 더부룩한 경우를 경험하는 경우가 있습니다. 저녁을 가볍게 먹는 것이 좋다고들 하지만, 사회생활을 유지하면서 저녁을 먹지 않는 것은 쉽지 않습니다. 그렇다면 아침에라도 조금 가볍게 먹으면 어떨까 싶은 생각을 하게 됩니다. 아침을 먹는 습관은 건강한 성인에게 권하는 경우가 많습니다. 특히 아침을 자주 거르는 사람에게는요. 그리고 아침을 가볍게 먹게 되면, 우리의 몸은 가벼운 식사에 맞춰서 기능을 해서, 우리의 몸은 가벼운 식사에 맞춰서 소화를 하게 됩니다. 그래서 만약 저녁에 너무 많이 먹거나, 식사 후에 활동량이 적게 된다면 속이 더부룩한 느낌을 받을 수 있습니다. 그리고 아침에 배가 고프지 않을 때는, 공복인데도, 잠을 자고 있거나, 활동량이 적어서, 우리의 위장이 일을 하지 않고 있을 경우입니다. 이럴 때는 아침을 먹고, 활동량을 늘려야, 우리의 위장도 자극을 받고 더 빨리 일을 할


In [ ]:
gen('인공지능을 공부하기 위해서는 어떻게 해야 할까?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:543: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1542: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 인공지능을 공부하기 위해서는 어떻게 해야 할까?

### 답변: 인공지능은 우리의 삶을 보다 편리하게 바꿀 수 있는 분야이므로 그 연구가 계속해서 진행될 것으로 내다보인다. 만약 수학과 공학에 관심이 있으면서, 인공지능에 특별한 지식을 얻고 싶다면 그 분야가 관련된 대학 및 교육기관에서 특별한 관심분야가 있는지 찾아보는 것을 추천한다. 대부분의 유명 대학에서는 인공지능과 관련된 수업을 제공하고 있기 때문에 이 점을 잘 알아본다면 보다 쉽게 대학 전공과목을 찾아볼 수 있을 것이다. 인공지능에는 여러 다른 분류가 있는데, 만약 특정 부류의 인공지능을 배우고자 한다면 이 종류의 인공지능을 많이 다루는 교육 및 공부 방법은 무엇인지에 관련된 사이트에서 찾아볼 수 있고, 예를 들어 의료 및 군사등 여러 분야에서의 인공지능이 유명한다. 마지막으로, 수학은 물론이고 코딩에도 능통해야만 인공지능을 배우기 위해 준비할 수 있다. 이러한 지식들은 컴퓨터를 다루는 모든 과정에 필요로 하는 것이다. 인공지능에 대해 보다 더 깊게 알고 싶다면, 수학은 물론이고, 프로그래밍에 대한 이해도 필요하게 될 것이다.########


**Reference**

https://github.com/Beomi/KoAlpaca?tab=readme-ov-file